In [1]:
import tensorflow as tf
import numpy as np
import sys

from sklearn.metrics import mean_squared_error
from keras.callbacks import LearningRateScheduler
#from sklearn.preprocessing import MinMaxScaler ##########

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
#sys.path.append('/Users/Think/Desktop/project')

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler, LinearPDEDataHandler

#Custom modules
from ann_framework import aux_functions

import aux_functions_stochastic
import analytic_functions
import loss_functions


Using TensorFlow backend.


In [2]:
#global variables

#deltas = [0.1, 0.1]
deltas = [0.01]

k = 1
c = 0.1
D = 1
num_fevals = 5

sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)

### Define tensorflow model

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y


def tf_ghazale(X):
    
    l2_lambda_regularization = 0.20
    #l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 2, activation=tf.nn.sigmoid, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_lambda_regularization), 
                         name="fc1")
    y = tf.layers.dense(A1, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_lambda_regularization), name="out")
    
    return y



def tf_model_yulin(X):
    
    A1 = tf.layers.dense(X, 32, activation=tf.nn.tanh, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),name="fc1")
    """
    dropout1 = tf.layers.dropout(inputs = A1, rate = 0.2)
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    dropout2 = tf.layers.dropout(inputs = A2, rate = 0.2)
    A3 = tf.layers.dense(A2, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc3")
    dropout3 = tf.layers.dropout(inputs = A3, rate = 0.2)
    A4 = tf.layers.dense(A3, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc4")
    dropout4 = tf.layers.dropout(inputs = A4, rate = 0.2)
    """
    y = tf.layers.dense(A1, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        name="out")
    #activation funcs
    
    return y


### Create Model TF

In [4]:
def tf_compiled_model(num_features, output_shape, num_fevals=1):
    tf.reset_default_graph()

    X, y = create_placeholders(num_features, output_shape)

    #y_pred = tf_model(X)
    #y_pred = tf_model_yulin(X)
    y_pred = tf_ghazale(X)
    
    #loss_function = loss_functions.squared_residual_function_wrapper2(k, c, D, deltas, num_fevals)
    loss_function =  loss_functions.linear_residual_function_wrapper(num_features, output_shape, deltas, num_fevals)
    cost, e = loss_function(X, y_pred, y)
    #reg_cost = tf.losses.get_regularization_loss()
    total_cost = e
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1, beta1=0.5).minimize(total_cost)

    return {'X_placeholder': X, 'y_placeholder': y, 'y_pred': y_pred, 'cost': cost, 'total_cost': total_cost, 'optimizer': optimizer}

### Create Tunable Model Tensorflow and assign data

In [5]:
"""

#scaler = MinMaxScaler()

dhandler_stochastic = OscillatorDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(2, 1, 5)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])

#Real phi function
p_real_test, phi_real_test = analytic_functions.real_p(tModel.X_test[:,0], tModel.X_test[:,1], sigma_x, sigma_y)
p_real_train, phi_real_train = analytic_functions.real_p(tModel.X_train[:,0], tModel.X_train[:,1], sigma_x, sigma_y)
p_real_crossVal, phi_real_crossVal = analytic_functions.real_p(tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], sigma_x, sigma_y)

tModel.y_test = phi_real_test
tModel.y_train = phi_real_train
tModel.y_crossVal = phi_real_crossVal


tModel.print_data()
"""

"\n\n#scaler = MinMaxScaler()\n\ndhandler_stochastic = OscillatorDataHandler()\n\n#(data_scaler = scaler)\n\nmodel = tf_compiled_model(2, 1, 5)\n\ntModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', \n                                        data_handler=dhandler_stochastic)\n\ntModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])\n\n#Real phi function\np_real_test, phi_real_test = analytic_functions.real_p(tModel.X_test[:,0], tModel.X_test[:,1], sigma_x, sigma_y)\np_real_train, phi_real_train = analytic_functions.real_p(tModel.X_train[:,0], tModel.X_train[:,1], sigma_x, sigma_y)\np_real_crossVal, phi_real_crossVal = analytic_functions.real_p(tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], sigma_x, sigma_y)\n\ntModel.y_test = phi_real_test\ntModel.y_train = phi_real_train\ntModel.y_crossVal = phi_real_crossVal\n\n\ntModel.print_data()\n"

## For linear PDE

In [6]:
#scaler = MinMaxScaler()

dhandler_stochastic = LinearPDEDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(1, 1, 3)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic, batch_size=8)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=0, boundaries=10, n=100)
#Real phi function
tModel.y_test = analytic_functions.real_y(tModel.X_test[:,0])
tModel.y_train = analytic_functions.real_y(tModel.X_train[:,0])
tModel.y_crossVal = analytic_functions.real_y(tModel.X_crossVal[:,0])

tModel.print_data()

Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

  `from __future__ import print_function`

Loading data for the first time
Reloading data due to parameter change
Loading data. Cros-Validation ratio 0.2
[[-10.        ]
 [ -9.89949749]
 [ -9.79899497]
 [ -9.69849246]
 [ -9.59798995]
 [ -9.497

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train model Tensorflow

In [7]:
tModel.epochs = 200
#lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches_linear

#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

#writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

    # cost_reg: total_cost -> e
    # cost: R

(24, 1)
(24, 1)
[[10.        ]
 [-5.27638191]
 [ 0.15075377]
 [-2.46231156]
 [ 5.57788945]
 [-6.48241206]
 [-7.18592965]
 [ 8.69346734]
 [10.01      ]
 [-5.26638191]
 [ 0.16075377]
 [-2.45231156]
 [ 5.58788945]
 [-6.47241206]
 [-7.17592965]
 [ 8.70346734]
 [ 9.99      ]
 [-5.28638191]
 [ 0.14075377]
 [-2.47231156]
 [ 5.56788945]
 [-6.49241206]
 [-7.19592965]
 [ 8.68346734]]
[[2.20264658e+04]
 [5.11088904e-03]
 [1.16271033e+00]
 [8.52376912e-02]
 [2.64512748e+02]
 [1.53011549e-03]
 [7.57164784e-04]
 [5.96382502e+03]
 [2.22478355e+04]
 [5.16225433e-03]
 [1.17439576e+00]
 [8.60943443e-02]
 [2.67171145e+02]
 [1.54549341e-03]
 [7.64774416e-04]
 [6.02376246e+03]
 [2.18072988e+04]
 [5.06003484e-03]
 [1.15114117e+00]
 [8.43895620e-02]
 [2.61880802e+02]
 [1.51489059e-03]
 [7.49630868e-04]
 [5.90448397e+03]]
18
Epoch: 0001 cost_reg= 5140528.662316878 cost= 0.332009715
(24, 1)
(24, 1)
[[-8.89447236]
 [ 0.05025126]
 [ 2.66331658]
 [-0.15075377]
 [-2.06030151]
 [-7.78894472]
 [-8.3919598 ]
 [ 9.698

Epoch: 0011 cost_reg= 5115387.058303621 cost= 126.328061422
(24, 1)
(24, 1)
[[ 8.19095477]
 [ 3.66834171]
 [ 2.16080402]
 [ 3.16582915]
 [-0.75376884]
 [ 0.15075377]
 [ 7.38693467]
 [-5.07537688]
 [ 8.20095477]
 [ 3.67834171]
 [ 2.17080402]
 [ 3.17582915]
 [-0.74376884]
 [ 0.16075377]
 [ 7.39693467]
 [-5.06537688]
 [ 8.18095477]
 [ 3.65834171]
 [ 2.15080402]
 [ 3.15582915]
 [-0.76376884]
 [ 0.14075377]
 [ 7.37693467]
 [-5.08537688]]
[[3.60816558e+03]
 [3.91868687e+01]
 [8.67811223e+00]
 [2.37083936e+01]
 [4.70589627e-01]
 [1.16271033e+00]
 [1.61474879e+03]
 [6.24873095e-03]
 [3.64442825e+03]
 [3.95807033e+01]
 [8.76532871e+00]
 [2.39466669e+01]
 [4.75319132e-01]
 [1.17439576e+00]
 [1.63097728e+03]
 [6.31153174e-03]
 [3.57226374e+03]
 [3.87969528e+01]
 [8.59176357e+00]
 [2.34724911e+01]
 [4.65907182e-01]
 [1.15114117e+00]
 [1.59868177e+03]
 [6.18655503e-03]]
18
Epoch: 0012 cost_reg= 5113179.519431644 cost= 143.419276343
(24, 1)
(24, 1)
[[ 1.25628141]
 [ 2.06030151]
 [-4.17085427]
 [-7.7

Epoch: 0021 cost_reg= 5093276.712513943 cost= 367.506823306
(24, 1)
(24, 1)
[[ 5.07537688]
 [ 9.1959799 ]
 [-5.47738693]
 [-7.78894472]
 [-7.38693467]
 [-3.5678392 ]
 [ 7.48743719]
 [ 4.87437186]
 [ 5.08537688]
 [ 9.2059799 ]
 [-5.46738693]
 [-7.77894472]
 [-7.37693467]
 [-3.5578392 ]
 [ 7.49743719]
 [ 4.88437186]
 [ 5.06537688]
 [ 9.1859799 ]
 [-5.48738693]
 [-7.79894472]
 [-7.39693467]
 [-3.5778392 ]
 [ 7.47743719]
 [ 4.86437186]]
[[1.60032494e+02]
 [9.85742147e+03]
 [4.18023868e-03]
 [4.14289844e-04]
 [6.19291377e-04]
 [2.82167587e-02]
 [1.78547039e+03]
 [1.30891909e+02]
 [1.61640848e+02]
 [9.95649021e+03]
 [4.22225077e-03]
 [4.18453526e-04]
 [6.25515359e-04]
 [2.85003419e-02]
 [1.80341467e+03]
 [1.32207394e+02]
 [1.58440144e+02]
 [9.75933849e+03]
 [4.13864461e-03]
 [4.10167591e-04]
 [6.13129325e-04]
 [2.79359973e-02]
 [1.76770467e+03]
 [1.29589513e+02]]
18
Epoch: 0022 cost_reg= 5091109.844530636 cost= 396.912415451
(24, 1)
(24, 1)
[[-1.35678392]
 [ 4.87437186]
 [-2.36180905]
 [ 8.9

Epoch: 0032 cost_reg= 5069257.547064198 cost= 779.380537245
(24, 1)
(24, 1)
[[-8.99497487]
 [ 3.06532663]
 [-7.18592965]
 [-0.35175879]
 [ 5.57788945]
 [-9.1959799 ]
 [-0.25125628]
 [ 4.47236181]
 [-8.98497487]
 [ 3.07532663]
 [-7.17592965]
 [-0.34175879]
 [ 5.58788945]
 [-9.1859799 ]
 [-0.24125628]
 [ 4.48236181]
 [-9.00497487]
 [ 3.05532663]
 [-7.19592965]
 [-0.36175879]
 [ 5.56788945]
 [-9.2059799 ]
 [-0.26125628]
 [ 4.46236181]]
[[1.24031515e-04]
 [2.14414643e+01]
 [7.57164784e-04]
 [7.03449778e-01]
 [2.64512748e+02]
 [1.01446408e-04]
 [7.77823004e-01]
 [8.75632867e+01]
 [1.25278052e-04]
 [2.16569546e+01]
 [7.64774416e-04]
 [7.10519566e-01]
 [2.67171145e+02]
 [1.02465961e-04]
 [7.85640256e-01]
 [8.84433124e+01]
 [1.22797380e-04]
 [2.12281182e+01]
 [7.49630868e-04]
 [6.96450336e-01]
 [2.61880802e+02]
 [1.00436999e-04]
 [7.70083536e-01]
 [8.66920175e+01]]
18
Epoch: 0033 cost_reg= 5067076.265382330 cost= 833.897181988
(24, 1)
(24, 1)
[[ 6.88442211]
 [ 0.45226131]
 [ 5.87939698]
 [-1.8

Epoch: 0043 cost_reg= 5045475.514519638 cost= 1318.301107672
(24, 1)
(24, 1)
[[ 4.17085427]
 [-1.15577889]
 [ 3.16582915]
 [-8.79396985]
 [-2.56281407]
 [ 8.09045226]
 [-5.47738693]
 [ 4.97487437]
 [ 4.18085427]
 [-1.14577889]
 [ 3.17582915]
 [-8.78396985]
 [-2.55281407]
 [ 8.10045226]
 [-5.46738693]
 [ 4.98487437]
 [ 4.16085427]
 [-1.16577889]
 [ 3.15582915]
 [-8.80396985]
 [-2.57281407]
 [ 8.08045226]
 [-5.48738693]
 [ 4.96487437]]
[[6.47707603e+01]
 [3.14812236e-01]
 [2.37083936e+01]
 [1.51644764e-04]
 [7.70875053e-02]
 [3.26316304e+03]
 [4.18023868e-03]
 [1.44730642e+02]
 [6.54217172e+01]
 [3.17976151e-01]
 [2.39466669e+01]
 [1.53168820e-04]
 [7.78622476e-02]
 [3.29595837e+03]
 [4.22225077e-03]
 [1.46185209e+02]
 [6.41262805e+01]
 [3.11679802e-01]
 [2.34724911e+01]
 [1.50135874e-04]
 [7.63204718e-02]
 [3.23069402e+03]
 [4.13864461e-03]
 [1.43290548e+02]]
18
Epoch: 0044 cost_reg= 5043292.811876979 cost= 1408.099339644
(24, 1)
(24, 1)
[[ 3.26633166]
 [ 6.18090452]
 [-5.87939698]
 [-6

...

Epoch: 0150 cost_reg= 4826270.803759681 cost= 11115.404391819
(24, 1)
(24, 1)
[[-6.7839196 ]
 [ 5.87939698]
 [-7.48743719]
 [ 0.15075377]
 [-6.58291457]
 [ 3.16582915]
 [-6.28140704]
 [-3.06532663]
 [-6.7739196 ]
 [ 5.88939698]
 [-7.47743719]
 [ 0.16075377]
 [-6.57291457]
 [ 3.17582915]
 [-6.27140704]
 [-3.05532663]
 [-6.7939196 ]
 [ 5.86939698]
 [-7.49743719]
 [ 0.14075377]
 [-6.59291457]
 [ 3.15582915]
 [-6.29140704]
 [-3.07532663]]
[[1.13182987e-03]
 [3.57593542e+02]
 [5.60076495e-04]
 [1.16271033e+00]
 [1.38381020e-03]
 [2.37083936e+01]
 [1.87076651e-03]
 [4.66386057e-02]
 [1.14320495e-03]
 [3.61187417e+02]
 [5.65705357e-04]
 [1.17439576e+00]
 [1.39771772e-03]
 [2.39466669e+01]
 [1.88956802e-03]
 [4.71073314e-02]
 [1.12056798e-03]
 [3.54035427e+02]
 [5.54503640e-04]
 [1.15114117e+00]
 [1.37004105e-03]
 [2.34724911e+01]
 [1.85215207e-03]
 [4.61745438e-02]]
18
Epoch: 0151 cost_reg= 4824315.768489838 cost= 11548.226721870
(24, 1)
(24, 1)
[[ 1.35678392]
 [-5.07537688]
 [ 1.45728643]
 [

Epoch: 0160 cost_reg= 4806837.022015254 cost= 12234.330895318
(24, 1)
(24, 1)
[[ 2.36180905]
 [-3.06532663]
 [ 0.45226131]
 [ 5.67839196]
 [-3.86934673]
 [ 8.89447236]
 [-9.59798995]
 [ 1.05527638]
 [ 2.37180905]
 [-3.05532663]
 [ 0.46226131]
 [ 5.68839196]
 [-3.85934673]
 [ 8.90447236]
 [-9.58798995]
 [ 1.06527638]
 [ 2.35180905]
 [-3.07532663]
 [ 0.44226131]
 [ 5.66839196]
 [-3.87934673]
 [ 8.88447236]
 [-9.60798995]
 [ 1.04527638]]
[[1.06101283e+01]
 [4.66386057e-02]
 [1.57186263e+00]
 [2.92478734e+02]
 [2.08719999e-02]
 [7.29155685e+03]
 [6.78650116e-05]
 [2.87276903e+00]
 [1.07167619e+01]
 [4.71073314e-02]
 [1.58766011e+00]
 [2.95418194e+02]
 [2.10817670e-02]
 [7.36483821e+03]
 [6.85470663e-05]
 [2.90164083e+00]
 [1.05045558e+01]
 [4.61745438e-02]
 [1.55622234e+00]
 [2.89568522e+02]
 [2.06643201e-02]
 [7.21900465e+03]
 [6.71897434e-05]
 [2.84418450e+00]]
18
Epoch: 0161 cost_reg= 4804902.831367810 cost= 12700.470013724
(24, 1)
(24, 1)
[[-7.38693467]
 [-0.85427136]
 [ 6.18090452]
 [

Epoch: 0177 cost_reg= 4773988.869370143 cost= 14394.100381533
(24, 1)
(24, 1)
[[-0.45226131]
 [ 8.59296482]
 [-2.46231156]
 [ 5.57788945]
 [ 5.87939698]
 [-3.86934673]
 [ 1.05527638]
 [-4.17085427]
 [-0.44226131]
 [ 8.60296482]
 [-2.45231156]
 [ 5.58788945]
 [ 5.88939698]
 [-3.85934673]
 [ 1.06527638]
 [-4.16085427]
 [-0.46226131]
 [ 8.58296482]
 [-2.47231156]
 [ 5.56788945]
 [ 5.86939698]
 [-3.87934673]
 [ 1.04527638]
 [-4.18085427]]
[[6.36187908e-01]
 [5.39358101e+03]
 [8.52376912e-02]
 [2.64512748e+02]
 [3.57593542e+02]
 [2.08719999e-02]
 [2.87276903e+00]
 [1.54390653e-02]
 [6.42581703e-01]
 [5.44778740e+03]
 [8.60943443e-02]
 [2.67171145e+02]
 [3.61187417e+02]
 [2.10817670e-02]
 [2.90164083e+00]
 [1.55942305e-02]
 [6.29857732e-01]
 [5.33991398e+03]
 [8.43895620e-02]
 [2.61880802e+02]
 [3.54035427e+02]
 [2.06643201e-02]
 [2.84418450e+00]
 [1.52854441e-02]]
18
Epoch: 0178 cost_reg= 4772103.243486828 cost= 14830.119611846
(24, 1)
(24, 1)
[[ 5.87939698]
 [-9.39698492]
 [-2.06030151]
 [

Epoch: 0188 cost_reg= 4752846.235838149 cost= 16140.879304144
(24, 1)
(24, 1)
[[ 0.65326633]
 [ 0.05025126]
 [ 6.48241206]
 [ 7.98994975]
 [-2.46231156]
 [ 4.57286432]
 [-7.18592965]
 [-3.66834171]
 [ 0.66326633]
 [ 0.06025126]
 [ 6.49241206]
 [ 7.99994975]
 [-2.45231156]
 [ 4.58286432]
 [-7.17592965]
 [-3.65834171]
 [ 0.64326633]
 [ 0.04025126]
 [ 6.47241206]
 [ 7.97994975]
 [-2.47231156]
 [ 4.56286432]
 [-7.19592965]
 [-3.67834171]]
[[1.92180785e+00]
 [1.05153527e+00]
 [6.53545438e+02]
 [2.95114866e+03]
 [8.52376912e-02]
 [9.68210396e+01]
 [7.57164784e-04]
 [2.55187524e-02]
 [1.94112234e+00]
 [1.06210337e+00]
 [6.60113678e+02]
 [2.98080819e+03]
 [8.60943443e-02]
 [9.77941072e+01]
 [7.64774416e-04]
 [2.57752201e-02]
 [1.90268554e+00]
 [1.04107232e+00]
 [6.47042552e+02]
 [2.92178424e+03]
 [8.43895620e-02]
 [9.58576541e+01]
 [7.49630868e-04]
 [2.52648366e-02]]
18
Epoch: 0189 cost_reg= 4750902.719417731 cost= 15892.076578564
(24, 1)
(24, 1)
[[-4.57286432]
 [ 6.08040201]
 [ 0.45226131]
 [

Epoch: 0198 cost_reg= 4733750.375372463 cost= 18045.671118312
(24, 1)
(24, 1)
[[-9.59798995]
 [-7.48743719]
 [ 1.45728643]
 [ 0.65326633]
 [-6.98492462]
 [ 8.79396985]
 [-6.08040201]
 [ 5.47738693]
 [-9.58798995]
 [-7.47743719]
 [ 1.46728643]
 [ 0.66326633]
 [-6.97492462]
 [ 8.80396985]
 [-6.07040201]
 [ 5.48738693]
 [-9.60798995]
 [-7.49743719]
 [ 1.44728643]
 [ 0.64326633]
 [-6.99492462]
 [ 8.78396985]
 [-6.09040201]
 [ 5.46738693]]
[[6.78650116e-05]
 [5.60076495e-04]
 [4.29429085e+00]
 [1.92180785e+00]
 [9.25733073e-04]
 [6.59435889e+03]
 [2.28725697e-03]
 [2.39220790e+02]
 [6.85470663e-05]
 [5.65705357e-04]
 [4.33744919e+00]
 [1.94112234e+00]
 [9.35036845e-04]
 [6.66063330e+03]
 [2.31024428e-03]
 [2.41624999e+02]
 [6.71897434e-05]
 [5.54503640e-04]
 [4.25156195e+00]
 [1.90268554e+00]
 [9.16521875e-04]
 [6.52874392e+03]
 [2.26449838e-03]
 [2.36840504e+02]]
18
Epoch: 0199 cost_reg= 4731801.896609094 cost= 18224.328923331
(24, 1)
(24, 1)
[[-0.95477387]
 [ 3.76884422]
 [-4.57286432]
 [

In [8]:
display_points = 20

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
X_test = tModel.X_crossVal
y_pred = tModel.y_predicted
y_real = tModel.y_crossVal
print("scores")

print(X_test)
print(y_real)

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(y_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)
print(sample_points)

y_real_sampled = y_real[sample_points]
y_pred_sampled = y_pred[sample_points]
X_sampled = X_test[sample_points,:]

print(y_real_sampled)

i = range(len(y_pred_sampled))


for x, y_real_display, y_pred_display in zip(X_sampled, y_real_sampled, y_pred_sampled):
    print('x {}, Real y {}, Predicted y {}'.format(x, y_real_display, y_pred_display))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

scores
[[-4.87437186]
 [-3.76884422]
 [ 9.09547739]
 [ 2.56281407]
 [-5.37688442]
 [ 2.7638191 ]
 [-6.38190955]
 [ 7.88944724]
 [-1.65829146]
 [-3.96984925]
 [-1.45728643]
 [ 1.15577889]
 [-8.19095477]
 [-6.68341709]
 [ 6.68341709]
 [ 9.49748744]
 [-1.95979899]
 [ 8.3919598 ]
 [-0.55276382]
 [-9.89949749]
 [ 7.68844221]
 [ 1.75879397]
 [ 1.95979899]
 [ 9.79899497]
 [ 9.89949749]
 [-4.47236181]
 [-7.88944724]
 [ 4.07035176]
 [ 6.38190955]
 [ 3.5678392 ]
 [ 7.78894472]
 [ 3.36683417]
 [-4.07035176]
 [ 4.27135678]
 [ 8.29145729]
 [ 6.28140704]]
[[7.63989164e-03]
 [2.30787218e-02]
 [8.91488283e+03]
 [1.29722709e+01]
 [4.62220035e-03]
 [1.58602994e+01]
 [1.69188913e-03]
 [2.66896820e+03]
 [1.90464118e-01]
 [1.88762786e-02]
 [2.32867319e-01]
 [3.17649661e+00]
 [2.77149143e-04]
 [1.25149420e-03]
 [7.99044857e+02]
 [1.33262018e+04]
 [1.40886737e-01]
 [4.41145479e+03]
 [5.75357427e-01]
 [5.01999018e-05]
 [2.18297130e+03]
 [5.80543165e+00]
 [7.09790021e+00]
 [1.80156297e+04]
 [1.99203577e+04]
 [

In [9]:
display_points = 50

"""
#Evaluate real model
X_test = tModel.X_crossVal

#Real phi function
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_test[:,0], X_test[:,1], sigma_x, sigma_y)

tModel.y_crossVal = phi_real

#p_real = np.ravel(p_real)
#phi_real = np.ravel(phi_real)

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
#phi_pred = np.ravel(tModel.y_predicted)
phi_pred = tModel.y_predicted

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)
"""

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
X_test = tModel.X_crossVal
phi_pred = tModel.y_predicted
phi_real = tModel.y_crossVal
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(phi_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)

phi_real_sampled = phi_real[sample_points]
phi_pred_sampled = phi_pred[sample_points]
X_sampled = X_test[sample_points,:]

i = range(len(phi_pred_sampled))


for i, phi_real_display, phi_pred_display in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_test[i], phi_real_display, phi_pred_display))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

scores
xy [-4.87437186], Real Phi [0.00763989], Predicted Phi [6.095154]
xy [-3.76884422], Real Phi [0.02307872], Predicted Phi [6.095154]
xy [9.09547739], Real Phi [8914.88282769], Predicted Phi [463.35117]
xy [2.56281407], Real Phi [12.97227088], Predicted Phi [7.5438232]
xy [-5.37688442], Real Phi [0.0046222], Predicted Phi [6.095154]
xy [2.7638191], Real Phi [15.86029943], Predicted Phi [9.131409]
xy [-6.38190955], Real Phi [0.00169189], Predicted Phi [6.095154]
xy [7.88944724], Real Phi [2668.9682038], Predicted Phi [463.35028]
xy [-1.65829146], Real Phi [0.19046412], Predicted Phi [6.095154]
xy [-3.96984925], Real Phi [0.01887628], Predicted Phi [6.095154]
xy [-1.45728643], Real Phi [0.23286732], Predicted Phi [6.095154]
xy [1.15577889], Real Phi [3.17649661], Predicted Phi [6.1029663]
xy [-8.19095477], Real Phi [0.00027715], Predicted Phi [6.095154]
xy [-6.68341709], Real Phi [0.00125149], Predicted Phi [6.095154]
xy [6.68341709], Real Phi [799.04485685], Predicted Phi [462.0998

In [10]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig1 = plt.figure(1)
ax1 = Axes3D(fig1)
pred_phi_plot = Axes3D.scatter(ax1, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_pred)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('phi(x, y)_pred')

plt.show()

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
fig2 = plt.figure(2)
ax2 = Axes3D(fig2)
real_phi_plot = Axes3D.scatter(ax2, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_real)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('phi(x, y)_real')

plt.show()

In [ ]:
fig3 = plt.figure(3)
ax3 = Axes3D(fig3)
pred_p_plot = Axes3D.scatter(ax3, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_pred)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('p(x, y)_pred')

plt.show()

In [ ]:
fig4 = plt.figure(4)
ax4 = Axes3D(fig4)
real_p_plot = Axes3D.scatter(ax4, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_real)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('p(x, y)_real')

plt.show()

In [ ]:
def real_derivatives(X, sigma_x, sigma_y):
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    first_order_dx = x1/(sigma_x**2)
    first_order_dy = x2/(sigma_y**2)
    second_order_dy = 1/(sigma_y**2)
    
    return first_order_dx, first_order_dy, second_order_dy

dx, dy, ddy = real_derivatives(X_test, sigma_x, sigma_y)

In [ ]:
dx, dy, ddy = analytic_functions.real_derivatives(tModel.X_crossVal, sigma_x, sigma_y)

#for i in range(len(dx)):
 #   print('dx {}, dy {}, ddy {}'.format(dx[i], dy[i], ddy[i]))